In [ ]:
import pyttsx3
import threading
import time
import queue

print("=" * 50)
print("TTS THREADING DIAGNOSTIC")
print("=" * 50)

# Test 1: Main thread (baseline - should work)
print("\n🧪 Test 1: Main thread speech...")
engine = pyttsx3.init()
engine.setProperty('rate', 180)
engine.say("Main thread working")
engine.runAndWait()
print("✅ Main thread: SUCCESS (you should have heard audio)")

# Test 2: Background thread (non-daemon)
print("\n🧪 Test 2: Background thread (non-daemon)...")
def bg_speech():
    e = pyttsx3.init()
    e.setProperty('rate', 180)
    e.say("Background thread")
    e.runAndWait()

t1 = threading.Thread(target=bg_speech)
t1.start()
t1.join()
print("✅ Background thread finished (did you hear audio?)")

# Test 3: Daemon thread (your current setup)
print("\n🧪 Test 3: DAEMON thread (your current setup)...")
def daemon_speech():
    e = pyttsx3.init()
    e.setProperty('rate', 180)
    e.say("Daemon thread test")
    e.runAndWait()

t2 = threading.Thread(target=daemon_speech, daemon=True)
t2.start()
time.sleep(2.0)  # Wait for speech to finish
print("✅ Daemon thread finished (did you hear audio?)")

# Test 4: Queue-based main-thread TTS (FIX)
print("\n🧪 Test 4: Queue-based TTS (FIX - runs in main thread)...")
response_q = queue.Queue()

def llm_sim():
    time.sleep(0.3)
    response_q.put("Queue based fix works!")

threading.Thread(target=llm_sim, daemon=True).start()

# Main thread polls queue and speaks
start = time.time()
while time.time() - start < 3.0:
    try:
        text = response_q.get_nowait()
        print(f"🗣️ Speaking from MAIN THREAD: {text}")
        engine.say(text)
        engine.runAndWait()
        print("✅ Queue-based TTS: SUCCESS")
        break
    except queue.Empty:
        time.sleep(0.01)
else:
    print("⚠️ No response received")

print("\n" + "=" * 50)
print("DIAGNOSIS:")
print("  • If Test 1 worked but Tests 2/3 failed → pyttsx3 requires MAIN THREAD on Windows")
print("  • Test 4 is the FIX: generate text in background, speak in main thread")
print("=" * 50)

🟢 Speech detected...
🟡 Pause detected...
🔴 Turn ended - queueing response
🗣️ Speaking: 'Hello there! This should play reliably.'
✅ Audio complete
✅ Test complete - did you hear the audio?


In [5]:
import pyttsx3
import subprocess
import sys
import time

print("=" * 60)
print("WINDOWS TTS DIAGNOSTIC")
print("=" * 60)

# Test 1: List available voices
print("\n🧪 Test 1: Checking available SAPI5 voices...")
try:
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    print(f"   Found {len(voices)} voice(s):")
    for i, v in enumerate(voices):
        print(f"     [{i}] {v.name} ({v.languages if hasattr(v, 'languages') else 'N/A'})")
    if not voices:
        print("   ⚠️ WARNING: No SAPI5 voices found!")
except Exception as e:
    print(f"   ❌ Error listing voices: {e}")

# Test 2: Explicitly test each voice
print("\n🧪 Test 2: Testing each voice (listen carefully)...")
if voices:
    for i, v in enumerate(voices[:2]):  # Test first 2 voices
        try:
            engine = pyttsx3.init()
            engine.setProperty('voice', v.id)
            engine.setProperty('rate', 150)
            print(f"   → Testing voice [{i}]: {v.name}")
            engine.say(f"Voice {i}")
            engine.runAndWait()
            time.sleep(0.5)
        except Exception as e:
            print(f"   ❌ Voice {i} failed: {e}")
else:
    print("   ⚠️ Skipping (no voices available)")

# Test 3: PowerShell speech (GUARANTEED to work on Windows 10/11)
print("\n🧪 Test 3: PowerShell speech (fallback)...")
try:
    cmd = 'Add-Type -AssemblyName System.Speech; $s=New-Object System.Speech.Synthesis.SpeechSynthesizer; $s.Speak("PowerShell speech works")'
    print("   → Speaking via PowerShell...")
    subprocess.run(["powershell", "-Command", cmd], 
                   stdout=subprocess.DEVNULL, 
                   stderr=subprocess.DEVNULL,
                   timeout=5)
    print("   ✅ PowerShell speech completed")
except Exception as e:
    print(f"   ❌ PowerShell failed: {e}")

# Test 4: Verify default audio device
print("\n🧪 Test 4: Checking default audio playback device...")
try:
    import sounddevice as sd
    devs = sd.query_devices()
    default_out = sd.default.device[1]  # Output device index
    print(f"   Default output device: {devs[default_out]['name']}")
    print(f"   Sample rate: {devs[default_out]['default_samplerate']} Hz")
except Exception as e:
    print(f"   ❌ Device check failed: {e}")

print("\n" + "=" * 60)
print("DIAGNOSIS:")
print("  • If Test 3 worked → Use PowerShell speech (100% reliable)")
print("  • If no voices found → Install Windows Speech Platform")
print("  • If voices exist but silent → Wrong audio device selected")
print("=" * 60)

WINDOWS TTS DIAGNOSTIC

🧪 Test 1: Checking available SAPI5 voices...
   Found 2 voice(s):
     [0] Microsoft David Desktop - English (United States) (['en-US'])
     [1] Microsoft Zira Desktop - English (United States) (['en-US'])

🧪 Test 2: Testing each voice (listen carefully)...
   → Testing voice [0]: Microsoft David Desktop - English (United States)
   → Testing voice [1]: Microsoft Zira Desktop - English (United States)

🧪 Test 3: PowerShell speech (fallback)...
   → Speaking via PowerShell...
   ✅ PowerShell speech completed

🧪 Test 4: Checking default audio playback device...
   Default output device: Speakers (Realtek USB2.0 Audio)
   Sample rate: 44100.0 Hz

DIAGNOSIS:
  • If Test 3 worked → Use PowerShell speech (100% reliable)
  • If no voices found → Install Windows Speech Platform
  • If voices exist but silent → Wrong audio device selected
